# Diffusion model generation

In this tutorial we will use a recently developed `Chemeleon` model to generate some crystal structures.

# Make sure you are running a GPU runtime

## Install the package

This may require running twice


In [2]:
!pip install chemeleon --quiet

## Load up the required functions

In [3]:
from chemeleon import Chemeleon
from chemeleon.visualize import Visualizer
from ase.io import write
import os

## Load the model

We load up a pre-trained model. We start with the composition mode. This model only takes compositions as a prompt. Note that the download and loading takes some time (2 - 5 minutes).

In [ ]:
%%time
composition_model = Chemeleon.load_composition_model()

## Set the generation parameters

Here we generate just one sample, to run quickly. But you can increase this later. Since we are using the composition only model, it can only take elements as a prompt.

In [17]:
# Set parameters
n_samples = 2
n_atoms = 8
prompt = "Li P S"

In [ ]:
%%time
# Generate crystal structures
atoms_list = composition_model.sample(prompt, n_atoms, n_samples)

In [ ]:
# Visualise
visualizer = Visualizer(atoms_list)
visualizer.view(index=0)

## Download structures

Get the structures and then use the `Alignn` notebook to calculate the total energy of the materials.

In [ ]:
import pickle
from google.colab import files

filehandler = open('diffusion-generated.pkl', 'wb')
pickle.dump(atoms_list, filehandler)


files.download('diffusion-generated.pkl')

## Exercise

Generate more samples of different chemistries

## Try generation with more conditions

We can use the `general_text_model` which allows us to use natural language to impose more conditions.

In [ ]:
# Load default model checkpoint (general text types)
chemeleon = Chemeleon.load_general_text_model()

In [27]:
# Set parameters
n_samples = 3
n_atoms = 56
text_inputs = "A crystal structure of LiMn2O4 with cubic symmetry"

In [ ]:
# Generate crystal structure
atoms_list = chemeleon.sample(text_inputs, n_atoms, n_samples)

In [ ]:
# Visualize the generated crystal structure
visualizer = Visualizer(atoms_list)
visualizer.view(index=0)

## View the generation trajectory

We can visualise the diffusion Langevin dynamcis that is used to generate the final structure from the initial sampling. To achive this we return the trajectory from the genertation process.

In the visulaisation note how the composition, the positions and the lattice are updated at the same time.

In [ ]:
n_samples = 1
n_atoms = 56
text_inputs = "A crystal structure of LiMn2O4 with cubic symmetry"

# Generate crystal structure with trajectory
trajectory = chemeleon.sample(text_inputs, n_atoms, n_samples, return_trajectory=True)

In [ ]:
# Visualize the trajectory
idx = 0
traj_0 = [t[idx] for t in trajectory][::10] + [trajectory[-1][idx]]
visualizer = Visualizer(traj_0, resolution=15)
visualizer.view_trajectory(duration=0.1)

## Exercise

Use `Chemeleon` to generate some of the materials that you generated previously with `CrystaLLM`. Then export these structures and compare the formation energies of the two different generative models.